# Analyzing Box Office Trends for 2018

## Devin Popa and Noah Bathras

### Introduction

As the star power, filming equipment, and post-production effects that make up modern films all improve, the budgets for these movies simultaneously increase dramatically.  This means it can be difficult for directors behind large studio films to make a return on their investment if they do not know what audiences want from movies.  In this project, we will be examining the role of a movie's genre, release date, target audience, and distributor in determining how much money it makes.  Then, we will compare the budgets and earnings of a sample of movies to find out how profitable a movie is based on each of these factors.

### Data Collection

The data we will be using can be found at https://www.the-numbers.com/market/2018/top-grossing-movies.  A request to the site will scrape the data initially, and BeautifulSoup will be used to parse the html of the page.  Several packages in the NumPy stack will be used to visualize and analyze the data.

In [1]:
# Necessary packages.  Add more as needed.
from bs4 import BeautifulSoup
import seaborn as sns
import pandas as pd
import numpy as np
import requests
import html5lib
import lxml     # open a new terminal and pip install lxml before running

# Make a request to the website to collect the data as html code.
req = requests.get('https://www.the-numbers.com/market/2018/top-grossing-movies')
soup = BeautifulSoup(req.text)

# Remove all links/tags from the entries
table = soup.find('table')
for a in table.findAll('a'):
    a.replaceWithChildren()

# Create a new DataFrame from the table variable
df_list = pd.read_html(str(table))
movie_df = df_list[0]
movie_df.columns = ['rank', 'movie', 'releaseDate', 'distributor', 
                    'genre', 'rating', 'annualGross', 'ticketsSold']
movie_df = movie_df.drop([0, 528, 529])     # drop unnecessary rows
movie_df.head()

# TODO
# 1. Fix weird characters showing up in movie titles
# 2. Basically all of the actual shit

,rank,movie,releaseDate,distributor,genre,rating,annualGross,ticketsSold
1,1,Black Panther,2/16/2018,Walt Disney,Action,PG-13,"$700,059,566",78044544
2,2,Avengers: Infinity War,4/27/2018,Walt Disney,Action,PG-13,"$678,815,482",75676196
3,3,Incredibles 2,6/15/2018,Walt Disney,Adventure,PG,"$608,546,638",67842434
4,4,Jurassic World: Fallen Kingdom,6/22/2018,Universal,Action,PG-13,"$416,769,345",46462580
5,5,Deadpool 2,5/18/2018,20th Century Fox,Action,R,"$318,491,426",35506290
